In [5]:
from __future__ import annotations

import sys
from pathlib import Path

import pandas as pd
from sklearn.datasets import load_breast_cancer

# Make sure the local Skyulf Core is importable whether this runs from repo root or from skyulf-core/
try:
    import skyulf  # noqa: F401
except ImportError:
    here = Path.cwd()
    candidates = [here, here / 'skyulf-core', here.parent, here.parent / 'skyulf-core']
    for c in candidates:
        if (c / 'skyulf' / '__init__.py').exists():
            sys.path.insert(0, str(c))
            break

from skyulf.pipeline import SkyulfPipeline

In [6]:
data = load_breast_cancer(as_frame=True)
df = data.frame.copy()
df = df.rename(columns={'target': 'label'})
df.head()

,mean radius,mean texture,mean perimeter,mean area,mean smoothness,mean compactness,mean concavity,mean concave points,mean symmetry,mean fractal dimension,...,worst texture,worst perimeter,worst area,worst smoothness,worst compactness,worst concavity,worst concave points,worst symmetry,worst fractal dimension,label
0,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,0.07871,...,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,0
1,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,0.05667,...,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,0
2,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,0.05999,...,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,0
3,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,0.09744,...,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,0
4,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,0.05883,...,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,0


In [7]:
feature_cols = [c for c in df.columns if c != 'label']

config = {
    'preprocessing': [
        {
            'name': 'impute',
            'transformer': 'SimpleImputer',
            'params': {'strategy': 'mean', 'columns': feature_cols},
        },
        {
            'name': 'scale',
            'transformer': 'StandardScaler',
            'params': {'columns': feature_cols},
        },
        {
            'name': 'split',
            'transformer': 'TrainTestSplitter',
            'params': {
                'test_size': 0.2,
                'random_state': 42,
                'shuffle': True,
                'stratify': True,
                'target_column': 'label',
            },
        },
    ],
    'modeling': {
        'type': 'logistic_regression',
        'node_id': 'model',
        'params': {'max_iter': 1000},
    },
}

pipe = SkyulfPipeline(config)
metrics = pipe.fit(df, target_column='label')
metrics

{'preprocessing': {'missing_counts': {'mean radius': 0,
   'mean texture': 0,
   'mean perimeter': 0,
   'mean area': 0,
   'mean smoothness': 0,
   'mean compactness': 0,
   'mean concavity': 0,
   'mean concave points': 0,
   'mean symmetry': 0,
   'mean fractal dimension': 0,
   'radius error': 0,
   'texture error': 0,
   'perimeter error': 0,
   'area error': 0,
   'smoothness error': 0,
   'compactness error': 0,
   'concavity error': 0,
   'concave points error': 0,
   'symmetry error': 0,
   'fractal dimension error': 0,
   'worst radius': 0,
   'worst texture': 0,
   'worst perimeter': 0,
   'worst area': 0,
   'worst smoothness': 0,
   'worst compactness': 0,
   'worst concavity': 0,
   'worst concave points': 0,
   'worst symmetry': 0,
   'worst fractal dimension': 0},
  'total_missing': 0,
  'fill_values': {'mean radius': 14.127291739894552,
   'mean texture': 19.289648506151142,
   'mean perimeter': 91.96903339191564,
   'mean area': 654.8891036906855,
   'mean smoothness'

In [8]:
pred = pipe.predict(df[feature_cols])
pred.head()

0    0
1    0
2    0
3    0
4    0
dtype: int32